In [3]:
from azureml.contrib.opendatasets import NycTlcYellow
from datetime import datetime
from dateutil import parser

end_date = parser.parse('2018-06-06')
start_date = parser.parse('2018-06-05')
nyc_tlc = NycTlcYellow(start_date=start_date, end_date=end_date)
nyc_tlc_df = nyc_tlc.to_spark_dataframe()

ModuleNotFoundError: No module named 'azureml.contrib'

In [ ]:
nyc_tlc_df.limit(5).show()

In [ ]:
group_df = nyc_tlc_df.groupBy("vendorID")
group_df.agg({'passengerCount':'sum'}).show()

In [4]:
import pprint
import pip
installed_packages = pip.get_installed_distributions()
installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
     for i in installed_packages])
pprint.pprint(installed_packages_list)

['absl-py==0.8.1',
 'adal==1.2.2',
 'alabaster==0.7.10',
 'altair==3.2.0',
 'applicationinsights==0.11.9',
 'asn1crypto==1.0.1',
 'astor==0.8.0',
 'astroid==1.4.9',
 'astropy==1.3.2',
 'attrs==19.2.0',
 'azure-common==1.1.23',
 'azure-graphrbac==0.61.1',
 'azure-mgmt-authorization==0.60.0',
 'azure-mgmt-containerregistry==2.8.0',
 'azure-mgmt-keyvault==2.0.0',
 'azure-mgmt-resource==5.1.0',
 'azure-mgmt-storage==4.2.0',
 'azure-storage-blob==2.1.0',
 'azure-storage-common==2.1.0',
 'azureml-automl-core==1.0.69',
 'azureml-core==1.0.69',
 'azureml-dataprep-native==13.1.0',
 'azureml-dataprep==1.1.29',
 'azureml-opendatasets==1.0.69',
 'azureml-pipeline-core==1.0.69',
 'azureml-pipeline-steps==1.0.69',
 'azureml-pipeline==1.0.69',
 'azureml-sdk==1.0.69',
 'azureml-telemetry==1.0.69',
 'azureml-train-automl==1.0.69',
 'azureml-train-core==1.0.69',
 'azureml-train-restclients-hyperdrive==1.0.69',
 'azureml-train==1.0.69',
 'babel==2.4.0',
 'backcall==0.1.0',
 'backports.shutil-get-terminal

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
blob_account_name = "datasetarcadia"
blob_container_name = "wideworldimporters"
blob_relative_path = "dimension_City/"
blob_sas_token = r'?sv=2018-03-28&ss=bfqt&srt=sco&sp=rl&se=2020-08-01T04:50:07Z&st=2019-07-23T20:50:07Z&spr=https&sig=62msikeBWwUcYmbAQI730n8ejbQH4QnJP0v2SYpYVzk%3D'

In [6]:
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)
print('Remote blob path: ' + wasbs_path)

Remote blob path: wasbs://wideworldimporters@datasetarcadia.blob.core.windows.net/dimension_City/

In [7]:
schema = StructType([ StructField("City_Key",IntegerType(),True),
              StructField("WWI_City_ID",IntegerType(),True),
              StructField("City",StringType(),True),
              StructField("State_Province",StringType(),True),
              StructField("Country",StringType(),True),
              StructField("Continent",StringType(),True),
              StructField("Sales_Territory",StringType(),True),
              StructField("Region",StringType(),True),
              StructField("Subregion",StringType(),True),
              StructField("Location",StringType(),True),
              StructField("Latest_Recorded_Population",LongType(),True),
              StructField("Valid_From",DateType(),True),
              StructField("Valid_To",DateType(),True),
              StructField("Lineage_Key",IntegerType(),True)
              ])

In [8]:
df = spark.read.option("header", "true") \
            .option("delimiter","|") \
            .schema(schema) \
            .csv(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')

Register the DataFrame as a SQL temporary view: source

In [9]:
df.show(10, truncate=False)

+--------+-----------+------------------+--------------+-------------+-------------+---------------+--------+----------------+------------------------------------+--------------------------+----------+----------+-----------+
|City_Key|WWI_City_ID|City              |State_Province|Country      |Continent    |Sales_Territory|Region  |Subregion       |Location                            |Latest_Recorded_Population|Valid_From|Valid_To  |Lineage_Key|
+--------+-----------+------------------+--------------+-------------+-------------+---------------+--------+----------------+------------------------------------+--------------------------+----------+----------+-----------+
|13079   |1448       |Atlantic Beach    |North Carolina|United States|North America|Southeast      |Americas|Northern America|POINT (-76.7402108 34.6990505)      |1495                      |2013-01-01|2013-01-01|1          |
|13080   |1449       |Atlantic City     |Wyoming       |United States|North America|Rocky Mountain |